#### 1_download sample street view images for image feature extraction

In [ ]:
import os
import requests
from tqdm import tqdm
from pathlib import Path
import json

# Set download path
download_dir = Path("./data/sample_images")
download_dir.mkdir(parents=True, exist_ok=True)

# API URL for image list
api_url = "https://huggingface.co/api/datasets/sijiey/Thermal-Affordance-Dataset/tree/main/sample_images"
base_url = "https://huggingface.co/datasets/sijiey/Thermal-Affordance-Dataset/resolve/main/sample_images/"

# Download function
def download_image(url, save_path):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    with open(save_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

# Get image list
print("Getting image list...")
try:
    response = requests.get(api_url)
    response.raise_for_status()
    file_list = response.json()
    image_names = [item['path'].split('/')[-1] for item in file_list 
                  if item['path'].endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(image_names)} images")
except Exception as e:
    print(f"Failed to get image list: {str(e)}")
    # If API call fails, use backup method
    print("Trying to download sample images directly...")
    image_names = [f"{i}.jpg" for i in range(1, 21)]  # Assuming 20 images named 1.jpg to 20.jpg

# Download all images
print("Starting image download...")
for img_name in tqdm(image_names):
    img_url = base_url + img_name
    save_path = download_dir / img_name
    
    # Skip download if file already exists
    if save_path.exists():
        print(f"File {img_name} already exists, skipping download")
        continue
        
    try:
        download_image(img_url, save_path)
    except Exception as e:
        print(f"Error downloading {img_name}: {str(e)}")

print("Download complete!")

#### 2_extract image features based on SVIs

In [ ]:
import urbancode as uc
import pandas as pd
import os

# Set image path
image_path = "./data/sample_images"  # Path to image folder

# Create a dataframe with image filenames
print("Getting image filenames...")
svi_df = uc.svi.filename(image_path)

# Extract segmentation features
print("\nExtracting segmentation features...")
svi_df = uc.svi.segmentation(svi_df, folder_path=image_path, filename_column='Filename')

# Extract object detection features
print("\nExtracting object detection features...")
svi_df = uc.svi.object_detection(svi_df, folder_path=image_path, filename_column='Filename')

# Extract color features
print("\nExtracting color features...")
svi_df = uc.svi.color(svi_df, folder_path=image_path, filename_column='Filename')

# Extract scene recognition features
print("\nExtracting scene recognition features...")
svi_df = uc.svi.scene_recognition(svi_df, folder_path=image_path, filename_column='Filename')

# Save results to CSV file
output_path = "./data/svi_features_results.csv"
svi_df.to_csv(output_path, index=False)
print(f"Results saved to: {output_path}")

# Display results preview
print("\nResults preview:")
print(svi_df.head())